In [ ]:
import requests
import re
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.chrome.service import Service
import json
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
#Util
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

def write_to_file(data):
    with open('benefits.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
labels = {
        "Fleksibilno radno vreme": "flex_hours",
        "Rad od kuće": "remote_work",
        "Plaćene obuke i treninzi": "paid_courses",
        "Rad preko agencije": "agency",
        "Privatno osiguranje": "insurance",
        "Zabavni i fitnes sadržaji": "entertainment",
        "Obezbeđena hrana": "lunch",
        "Obezbeđen parking": "parking_lot",
        "Deljenje profita": "profit_sharing",
        "Akcije kompanije": "company_shares",
}

In [ ]:
def get_review_count():
    elements_to_hover = driver.find_elements(By.CLASS_NAME, "benefits-row")
    for element in elements_to_hover:
        hover = ActionChains(driver).move_to_element(element)
        hover.perform()
        content = driver.find_element(By.CSS_SELECTOR, ".ui.top.left.inverted.popup.transition.visible").get_attribute("innerHTML")
        print(content)
        total = content.split(" ")[-2]
        print(total.isnumeric(), total)
        if total.isnumeric():
            return total
    
    return 0

def get_benefits(url):
    driver.get(url)
    time.sleep(2)
    company_name = company_url.split("/")[-2]
    json = {
        "company": company_name
    }

    #no info
    if (check_exists_by_xpath(driver,"//div[@class='ui centered stackable grid']//div[@class='ui info message']")):
        return {}
    
    categories = driver.find_elements(By.XPATH, "//div[@class='x-container']//div[@class='x-col']//div[@class='pb-10 x-row']")
    for category in categories:
        category_name = category.find_element(By.XPATH, ".//span[@class='font-size-09']").get_attribute("innerHTML")
        if (check_exists_by_xpath(category,".//span[@class='benefits-row']/span")):
           value = category.find_element(By.XPATH, ".//span[@class='benefits-row']/span").get_attribute("innerHTML")
        else:
            value = ""
        json[labels[category_name]] = value
    json["review_count"] = get_review_count()

    return json

In [ ]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
file = open('links.txt', 'r')
lines = file.readlines()

ln = []
for line in lines:
    line = line.strip()
    line = line.replace('o-kompaniji', 'plate-i-benefiti')
    ln.append(line)

In [ ]:
data = []
for company_url in ln:
    if not company_url:
        pass
    benefits = get_benefits(company_url)
    if benefits:
        data.append(benefits)

write_to_file(data)


In [18]:
import pandas as pd
import numpy as np

df = pd.read_json('benefits.json')
df = df.replace(r'^\s*$', np.NaN, regex=True)
df

,company,flex_hours,remote_work,paid_courses,agency,insurance,entertainment,lunch,parking_lot,profit_sharing,company_shares,review_count
0,iten-engineering,2,1,1,NaN,2,2,2,NaN,NaN,NaN,2
1,doob-innovation-studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,wireless-media,8,12,6,3,14,NaN,3,NaN,NaN,NaN,17
3,merkle,20,23,14,NaN,24,15,NaN,NaN,NaN,NaN,24
4,sks365,6,3,4,3,7,7,2,NaN,1,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...
328,softmetrix,2,2,1,NaN,NaN,1,NaN,NaN,NaN,NaN,2
329,mangonel,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
330,quectel,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,3
331,prodyna,3,1,2,1,2,2,3,3,NaN,NaN,3


In [19]:
sum([True for idx,row in df.iterrows() if any(row.isnull())])

315

In [32]:
df2 = df.apply(lambda x: sum(x.isnull().values), axis = 1) # For rows
df2 = pd.DataFrame(df2, columns = ['zero_num'])
df2[df2.zero_num > 5].shape[0]


115